In [ ]:
import requests
year = 2022
day = 16

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open('cookie.txt', 'r') as file:
    session = file.read()

headers = {'cookie': f"session={session}"}

with open(f'day_{day}_input.txt', 'wb') as file:
    file.write(requests.get(url, headers=headers).content)
    
import os
if os.path.exists(f"day_{day}_test.txt"):
    test = [x.strip() for x in open(f'day_{day}_test.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
if os.path.exists(f"day_{day}_test_2.txt"):
    test_2 = [x.strip() for x in open(f'day_{day}_test_2.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
lines = [x.strip() for x in open(f'day_{day}_input.txt', 'r')]
for line in lines[:10]:
    print(line.strip())

In [457]:
import re

run_test = False

if run_test:
    case = test
else:
    case = lines
    
V = {}
L = []
for line in case:
    pattern = r"Valve (\S+) has flow rate=(\d+); tunnels? leads? to valves? ([\S, ]+)"
    groups = re.findall(pattern,line)
    s = groups[0][0]
    p = int(groups[0][1])
    ts = [x.strip() for x in groups[0][2].split(',')]
    V[s] = p
    for t in ts:
        L.append((s,t))

In [345]:
from pprint import pprint
#print(V)

# At each step, options are:
    # move, open
    
DP = dict()

num_lookups = 0
num_calcs = 0

def advance_time(state, current_pressure_released):
    global num_calcs, num_lookups
    position = state[0]
    time = state[1]
    current_open_valves = state[2]
    
    if time == 30:
        #assert False
        #print(current_pressure_released)
        return current_pressure_released
    
    if state in DP:
        num_lookups+=1
        return DP[state]
    else:
        num_calcs+=1
        current_pressure_released += sum([V[v] for v in current_open_valves])
        #print(state, current_pressure_released)

        # Move
        for l in [l[1] for l in L if l[0] == position]:
            n_state = (l, time+1, current_open_valves)
            advance_time(n_state, current_pressure_released)
            #DP[n_state] = current_pressure_released


        # Open
        for l in [l[0] for l in L if l[0] == position and l[0] not in current_open_valves and V[l[0]] > 0]:
            n_state = (position, time+1, current_open_valves + tuple([l]))
            advance_time(n_state, current_pressure_released)
        
        DP[state] = current_pressure_released
        #return DP[state]

advance_time(('AA',0,()),0)
# Too slow
#pprint(DP)
print(num_lookups,num_calcs)
print(max(DP.values()))

76198 73640
1651


In [443]:
print([v for v in V if V[v] > 0])

['AT', 'GQ', 'OC', 'TR', 'VK', 'YL', 'ES', 'XN', 'EY', 'YA', 'AZ', 'WZ', 'UI', 'JT', 'RK']


In [ ]:
from pprint import pprint
from collections import deque
    
DP = dict()

MAX_DURATION = 26

def advance_time(state, current_pressure_released):
    positions = state[0]
    time = state[1]
    current_open_valves = state[2]
        
    current_pressure_released = current_pressure_released + sum([V[v] for v in current_open_valves])
    if state in DP:
        if current_pressure_released <= DP[state]:
            return DP[state]
    
    DP[state] = current_pressure_released
    
    #all valves open - sprint to end.
    if len(current_open_valves) == len([v for v in V if V[v] > 0]):
        for i, t in enumerate(range(time+1,MAX_DURATION+1)):
            n_state = (positions, t, current_open_valves)
            if n_state not in DP:
                DP[n_state] = current_pressure_released + i * sum([V[v] for v in current_open_valves])
        return DP[state]

    if time == DURATION:
        return DP[state]
    
    searches = list()

    # both open
    if V[positions[0]] > 0 and V[positions[1]] > 0:
        n_state = (positions, time+1, tuple(sorted(set(current_open_valves + positions))))
        searches.append(n_state)

    #you move, elephant opens
    for i in [l[1] for l in L if l[0] == positions[0]]:
        for j in [l[0] for l in L if l[0] == positions[1] and V[positions[1]] > 0]:
            n_state = ((i,positions[1]), time+1, tuple(sorted(set(current_open_valves + tuple([j])))))
            searches.append(n_state)

    # elephant moves, you open
    for i in [l[0] for l in L if l[0] == positions[0] and V[positions[0]] > 0]:
        for j in [l[1] for l in L if l[0] == positions[1]]:
            n_state = ((positions[0],j), time+1, tuple(sorted(set(current_open_valves + tuple([i])))))
            searches.append(n_state)


    # both move
    for i in [l[1] for l in L if l[0] == positions[0]]:
        for j in [l[1] for l in L if l[0] == positions[1]]:
            n_state = ((i,j), time+1, tuple(sorted(set(current_open_valves))))
            searches.append(n_state)

    # both stay
    n_state = (positions, time+1, tuple(sorted(set(current_open_valves))))
    advance_time(n_state, current_pressure_released)
    searches.append(n_state)

    for search in set(searches):#sorted(searches, key= lambda x: sum([V[v] for v in current_open_valves if v in x])):            
        advance_time(search, current_pressure_released)
        
    return DP[state]
        

DURATION = 26
advance_time((('AA','AA'), 0, ()),0)
#advance_time((('AA', 'EE'), 11, ('BB', 'CC', 'DD', 'EE', 'HH', 'JJ')),573)

#print(max(DP.values()))

for i in range(1,26+1):
    max_day = 0
    max_key = None
    for k,v in DP.items():
        if k[1] == i and v >= max_day:
            max_day = v
            max_key = k
    print(f"{i}: {max_day}, {max_key}")
# 2298 too high
# 2113 too low

In [386]:
from pprint import pprint
from collections import deque

    
DP = {(('AA','AA'),0,()):0}

def advance_time(state, current_pressure_released):
    positions = state[0]
    time = state[1]
    current_open_valves = state[2]

    current_pressure_released = current_pressure_released + sum([V[v] for v in current_open_valves])

    if state not in DP:
        DP[state] = current_pressure_released
        
    if current_pressure_released > DP[state]:
        DP[state] = current_pressure_released

    if time == DURATION:
        return []# DP[state]

    searches = list()

    if V[positions[0]] > 0 and V[positions[1]] > 0:
        n_state = (positions, time+1, tuple(sorted(set(current_open_valves + positions))))
        searches.append(n_state)

    #you move, elephant opens
    for i in [l[1] for l in L if l[0] == positions[0]]:
        for j in [l[0] for l in L if l[0] == positions[1] and V[l[0]] > 0]:
            n_state = ((i,positions[1]), time+1, tuple(sorted(set(current_open_valves + tuple([j])))))
            searches.append(n_state)

    # elephant moves, you open
    for i in [l[0] for l in L if l[0] == positions[0] and V[l[0]] > 0]:
        for j in [l[1] for l in L if l[0] == positions[1]]:
            n_state = ((positions[0],j), time+1, tuple(sorted(set(current_open_valves + tuple([i])))))
            searches.append(n_state)


    # both move
    for i in [l[1] for l in L if l[0] == positions[0]]:
        for j in [l[1] for l in L if l[0] == positions[1]]:
            n_state = ((i,j), time+1, tuple(sorted(set(current_open_valves))))
            searches.append(n_state)

        
        # both stay
        #n_state = (positions, time+1, tuple(sorted(set(current_open_valves))))
        #advance_time(n_state, current_pressure_released)
        #searches.append(n_state)
#         print(len(searches))
#         pprint(searches)
        

    #return [search for search in sorted(searches, key= lambda x: sum([V[v] for v in current_open_valves if v in x])), current_pressure_released]
    return set(searches), current_pressure_released

DURATION = 12

best = {(('AA','AA'),0,()):0}
for i in range(DURATION):
    to_search = []
    for state in best:
        searches = advance_time(state,0)
        to_search.extend(searches[0])
        #pprint(searches)
    best = to_search.copy()
    

for i in range(1,DURATION):
    max_day = 0
    max_key = None
    for k,v in DP.items():
        if k[1] == i and v >= max_day:
            max_day = v
            max_key = k
    print(f"{i}: {max_day}, {max_key}")

KeyboardInterrupt: 

In [442]:
from collections import defaultdict
from collections import deque
from functools import cache
import itertools as it
import os

ValveList = []
with open("day_16_input.txt", "r") as data:
    for t in data:
        Line = t.strip().split()
        Name, Rate, Tunnels = Line[1], int(Line[4][5:-1]), tuple(Line[9:])
        NewTuple = (Name, Rate, Tunnels)
        ValveList.append(NewTuple)

NumberValves = len(ValveList)
ValveNameDict = defaultdict()
ValveIntDict = defaultdict()
for n, v in enumerate(ValveList):
    Name, Rate, Tunnels = v
    Tunnels = list(Tunnels)
    for m, t in enumerate(Tunnels):
        if len(t) > 2:
            Tunnels[m] = t[:-1]
    Tunnels = tuple(Tunnels)
    ValveNameDict[Name] = (n, Rate, Tunnels)
    ValveIntDict[n] = Rate
Psuedocache = set()

def ParseValve(Minutes, Location, ActiveValveInt, NumberValvesTotal, PastPressure):
    Index, Rate, Tunnels = ValveNameDict[Location]
    IndexBinary = bin(Index)[2:]
    ValveBinary = bin(ActiveValveInt)[2:]
    while len(IndexBinary) < 6:
        IndexBinary = "0" + IndexBinary
    while len(ValveBinary) < NumberValvesTotal:
        ValveBinary = "0" + ValveBinary
    NewBinary = IndexBinary + ValveBinary
    StateInt = int(NewBinary, 2)
    if StateInt in Psuedocache:
        return 0
    Psuedocache.add(StateInt)

    ActivePressure = 0
    for n, v in enumerate(ValveBinary):
        if v == "1":
            NewRate = ValveIntDict[n]
            ActivePressure += NewRate
    TotalPressure = ActivePressure + PastPressure

    if Minutes >= 30:
        return TotalPressure
    
    if Rate > 0 and ValveBinary[Index] == "0":
        NewActiveBinary = list(ValveBinary)
        NewActiveBinary[Index] = "1"
        NewActiveInt = int("".join(NewActiveBinary), 2)
        NewTuple = (Minutes+1, Location, NewActiveInt, TotalPressure)
        PathQueue.append(NewTuple)
    
    for t in Tunnels:
        NewTuple = (Minutes+1, t, ActiveValveInt, TotalPressure)
        PathQueue.append(NewTuple)
    
    return TotalPressure + (30-Minutes)*ActivePressure

PathQueue = deque()
PathQueue.append((1, "AA", 0, 0))
Cycles = 0
Part1Answer = 0
while PathQueue:
    Minutes, Location, ValveInt, CurrentPressure = PathQueue.popleft()
    NextEntry = ParseValve(Minutes, Location, ValveInt, NumberValves, CurrentPressure)
    if NextEntry > Part1Answer:
        Part1Answer = NextEntry
    Cycles += 1

##########Part 2

P2IntDict = defaultdict()
P2NameDict = defaultdict()
WorkingValveSet = set()
WorkingValveList = []
Index = 0
for v in ValveList:
    Name, Rate, Tunnels = v
    if Rate == 0:
        continue
    WorkingValveSet.add(Name)
    P2IntDict[Index] = Rate
    P2NameDict[Name] = (Index, Rate)
    WorkingValveList.append(Name)
    Index += 1


PathLengthDict = defaultdict()
def BFS(Valve):
    VisitedSet = set()
    VisitedSet.add(Valve)
    Queue = deque()
    Queue.append((0, Valve))
    while Queue:
        Length, Position = Queue.popleft()
        _, _, Tunnels = ValveNameDict[Position]
        if Position in WorkingValveSet and Length != 0:
            PathLengthDict[(Valve, Position)] = Length
        if Position == "AA":
            PathLengthDict[(Position, Valve)] = Length
        for t in Tunnels:
            if t in VisitedSet:
                continue
            Queue.append((Length+1, t))
            VisitedSet.add(t)

for v in WorkingValveSet:
    BFS(v)
StatesSeen = 0

@cache
def P2Parse(Minutes, Location, ArrivalTime, ActiveValveInt, PlayersRemaining):
    global StatesSeen

    if Minutes == 0 and PlayersRemaining == 1:
        return P2Parse(26, "AA", 0, ActiveValveInt, 0)
    if Minutes == 0 and PlayersRemaining == 0:
        return 0
    
    StatesSeen += 1
    if StatesSeen % 100000 == 4:
        print(StatesSeen)
    if ArrivalTime > 0:
        return P2Parse(Minutes-1, Location, ArrivalTime-1, ActiveValveInt, PlayersRemaining)
    
    ValveBinary = bin(ActiveValveInt)[2:]
    while len(ValveBinary) < len(WorkingValveSet):
        ValveBinary = "0" + ValveBinary
    Score = 0
    if Location != "AA":
        Index, Rate = P2NameDict[Location]
        Score = Minutes*Rate
        NewValveInt = list(ValveBinary)
        NewValveInt[Index] = "1"
        NewValveInt = int("".join(NewValveInt), 2)
    
    Scoring = [Score]
    
    for v in WorkingValveList:
        NewIndex, NewRate = P2NameDict[v]
        if v == Location:
            continue
        if ValveBinary[NewIndex] == "1":
            continue
        NewLength = PathLengthDict[Location, v]
        if NewLength > Minutes:
            continue
        if Location == "AA":
            NewScore = Score + P2Parse(Minutes-1, v, NewLength, ActiveValveInt, PlayersRemaining)
            Scoring.append(NewScore)
        else:
            NewScore = Score + P2Parse(Minutes-1, v, NewLength, NewValveInt, PlayersRemaining)
            Scoring.append(NewScore)
    
    if PlayersRemaining == 1 and Location != "AA":
        Scoring.append(Score + P2Parse(26, "AA", 0, NewValveInt, 0))
    return max(Scoring)


print(PathLengthDict)


Part2Answer = P2Parse(26, "AA", 0, 0, 1)

print(f"{Part1Answer = }")
print(f"{Part2Answer = }")

defaultdict(None, {('UI', 'RK'): 2, ('UI', 'XN'): 2, ('AA', 'UI'): 2, ('UI', 'OC'): 3, ('UI', 'AT'): 3, ('UI', 'EY'): 5, ('UI', 'VK'): 5, ('UI', 'ES'): 5, ('UI', 'TR'): 6, ('UI', 'JT'): 7, ('UI', 'AZ'): 7, ('UI', 'GQ'): 7, ('UI', 'WZ'): 8, ('UI', 'YL'): 9, ('UI', 'YA'): 10, ('OC', 'XN'): 2, ('OC', 'VK'): 3, ('OC', 'UI'): 3, ('AA', 'OC'): 3, ('OC', 'AT'): 3, ('OC', 'AZ'): 5, ('OC', 'GQ'): 5, ('OC', 'RK'): 5, ('OC', 'ES'): 5, ('OC', 'WZ'): 6, ('OC', 'EY'): 6, ('OC', 'TR'): 6, ('OC', 'YL'): 7, ('OC', 'YA'): 8, ('OC', 'JT'): 8, ('EY', 'JT'): 2, ('EY', 'AZ'): 2, ('EY', 'RK'): 3, ('AA', 'EY'): 3, ('EY', 'GQ'): 4, ('EY', 'VK'): 4, ('EY', 'UI'): 5, ('EY', 'YA'): 5, ('EY', 'XN'): 6, ('EY', 'OC'): 6, ('EY', 'YL'): 6, ('EY', 'WZ'): 7, ('EY', 'AT'): 8, ('EY', 'ES'): 10, ('EY', 'TR'): 11, ('XN', 'OC'): 2, ('XN', 'UI'): 2, ('AA', 'XN'): 3, ('XN', 'AT'): 3, ('XN', 'RK'): 3, ('XN', 'VK'): 5, ('XN', 'ES'): 5, ('XN', 'EY'): 6, ('XN', 'TR'): 6, ('XN', 'AZ'): 7, ('XN', 'GQ'): 7, ('XN', 'WZ'): 8, ('XN', 'J